In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_1207172/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
    
    def has_minimum_requirements(self, df, sort_by="r2", boundary=0.75):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.discard_below_average(sort_by="r2")
        self.discard_below_average(sort_by="r2_vt")
        self.discard_high_standard_deviation()
        self.results_df.to_excel(f"better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer()
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt,Architecture
0,model_1_9_0,0.947092,0.717500,0.938272,0.840206,6.380669,24.023575,7.361326,16.182520,"Hidden Size=[70, 35], regularizer=0.02, learni..."
1,model_1_9_1,0.946897,0.711629,0.938264,0.837592,6.404233,24.522850,7.362209,16.447250,"Hidden Size=[70, 35], regularizer=0.02, learni..."
2,model_1_9_2,0.946661,0.706036,0.937854,0.834878,6.432745,24.998528,7.411098,16.722105,"Hidden Size=[70, 35], regularizer=0.02, learni..."
3,model_1_9_3,0.946377,0.700726,0.937004,0.832046,6.466902,25.450079,7.512527,17.008879,"Hidden Size=[70, 35], regularizer=0.02, learni..."
4,model_1_9_4,0.946043,0.695675,0.935696,0.829076,6.507195,25.879580,7.668561,17.309706,"Hidden Size=[70, 35], regularizer=0.02, learni..."
...,...,...,...,...,...,...,...,...,...,...
381,model_3_8_1,0.831191,0.602180,0.774443,0.755715,20.358423,30.786055,44.511063,37.244873,"Hidden Size=[70, 35], regularizer=0.1, learnin..."
397,model_3_8_0,0.829187,0.600732,0.779783,0.758578,20.600067,30.898115,43.457211,36.808270,"Hidden Size=[70, 35], regularizer=0.1, learnin..."
423,model_7_1_0,0.824429,0.698968,0.729150,0.743829,21.173862,22.253078,45.396252,33.143997,"Hidden Size=[30, 15], regularizer=0.1, learnin..."
425,model_5_5_0,0.824327,0.947833,0.964917,0.971224,21.186239,3.600373,5.294904,4.397810,"Hidden Size=[30, 15], regularizer=0.02, learni..."
